In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

embedding = OllamaEmbeddings(model="qwen2.5")
llm = ChatOllama(
    model="qwen2.5",
    temperature=0.0,
    max_tokens=100,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.0,
)

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [3]:
# 加载向量库，其中包含了所有课程材料的 Embedding。
from langchain.vectorstores import Chroma
import panel as pn  # GUI

# pn.extension()

persist_directory = "../docs/chroma/matplotlib"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

question = "这门课的主要内容是什么？"
docs = vectordb.similarity_search(question, k=3)
print(len(docs))

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_72815/1153810359.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


3


In [6]:
# pretty_print_docs(docs)

In [ ]:
# 构建 prompt
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 运行 chain
from langchain.chains import RetrievalQA

question = "这门课的主题是什么？"
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)


result = qa_chain({"query": question})
print(result["result"])